In [1]:
!pip install tensorflow==2.18
!pip install --upgrade pip
!pip install --upgrade tensorflow tensorflow_datasets

   ---------------------------------------- 0.0/390.2 MB ? eta -:--:--
   ---------------------------------------- 1.6/390.2 MB 8.4 MB/s eta 0:00:47
   ---------------------------------------- 3.4/390.2 MB 8.4 MB/s eta 0:00:47
   ---------------------------------------- 4.2/390.2 MB 9.7 MB/s eta 0:00:40
   ---------------------------------------- 4.2/390.2 MB 9.7 MB/s eta 0:00:40
    --------------------------------------- 6.0/390.2 MB 5.8 MB/s eta 0:01:07
    --------------------------------------- 7.3/390.2 MB 6.0 MB/s eta 0:01:04
    --------------------------------------- 9.4/390.2 MB 6.6 MB/s eta 0:00:58
   - -------------------------------------- 11.5/390.2 MB 7.0 MB/s eta 0:00:55
   - -------------------------------------- 12.3/390.2 MB 6.6 MB/s eta 0:00:58
   - -------------------------------------- 13.6/390.2 MB 6.8 MB/s eta 0:00:56
   - -------------------------------------- 15.7/390.2 MB 7.0 MB/s eta 0:00:54
   - -------------------------------------- 17.3/390.2 MB 6.8 MB/s 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.4 which is incompatible.
tensorflow-intel 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.20.0 which is incompatible.


In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
from sklearn.preprocessing import StandardScaler

def featurize_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(2050)

    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hbd = Descriptors.NumHDonors(mol)
    hba = Descriptors.NumHAcceptors(mol)
    tpsa = Descriptors.TPSA(mol)

    fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    fp = np.array(fp)

    return np.concatenate([[mw, logp, hbd, hba, tpsa], fp])
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

X_train = np.vstack(train["SMILES"].apply(featurize_smiles)).astype('float32')
X_test = np.vstack(test["SMILES"].apply(featurize_smiles)).astype('float32')
y_train = train["Tm"].values.astype("float32")


In [16]:
num_features = X_train.shape[1]
batch_size = 128
ds_train = tf.data.Dataset.from_tensor_slices((X_train.astype('float32'), y_train.astype('float32')))
ds_train = ds_train.shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

n_hidden_1 = 256
n_hidden_2 = 128
learning_rate = 0.001

initializer = tf.initializers.GlorotUniform()

weights = {
    "h1": tf.Variable(initializer([num_features, n_hidden_1], dtype=tf.float32)),
    "h2": tf.Variable(initializer([n_hidden_1, n_hidden_2], dtype=tf.float32)),
    "out": tf.Variable(initializer([n_hidden_2, 1], dtype=tf.float32))
}

biases = {
    "b1": tf.Variable(tf.zeros([n_hidden_1], dtype=tf.float32)),
    "b2": tf.Variable(tf.zeros([n_hidden_2], dtype=tf.float32)),
    "out": tf.Variable(tf.zeros([1], dtype=tf.float32))
}

# ------------------------------------------------------
# 6) Modell (wie im MNIST Code)
# ------------------------------------------------------
def neural_net(x):
    layer1 = tf.nn.relu(tf.matmul(x, weights["h1"]) + biases["b1"])
    layer2 = tf.nn.relu(tf.matmul(layer1, weights["h2"]) + biases["b2"])
    out = tf.matmul(layer2, weights["out"]) + biases["out"]  # LINEARER OUTPUT
    return out  # shape: (batch, 1)

# ------------------------------------------------------
# 7) Loss + Optimizer
# ------------------------------------------------------
def mse_loss(y_pred, y_true):
    y_true = tf.reshape(y_true, (-1, 1))
    return tf.reduce_mean((y_true - y_pred) ** 2)

optimizer = tf.optimizers.Adam(learning_rate)

print("x dtype:", bx.dtype)
print("weights dtype:", weights["h1"].dtype)

# ------------------------------------------------------
# 8) Trainingsschritt
# ------------------------------------------------------
def run_optimization(x, y):
    with tf.GradientTape() as tape:
        pred = neural_net(x)
        loss = mse_loss(pred, y)

    trainable_vars = list(weights.values()) + list(biases.values())
    gradients = tape.gradient(loss, trainable_vars)
    optimizer.apply_gradients(zip(gradients, trainable_vars))
    return loss


# ------------------------------------------------------
# 9) Training
# ------------------------------------------------------
epochs = 20

for epoch in range(1, epochs + 1):
    batch_losses = []
    for bx, by in ds_train:
        loss = run_optimization(bx, by)
        batch_losses.append(loss.numpy())

    print(f"Epoch {epoch:02d}  Loss: {np.mean(batch_losses):.4f}")

# ------------------------------------------------------
# 10) Vorhersage für Testdaten
# ------------------------------------------------------
pred_test = neural_net(X_test).numpy().flatten()

print("\nBeispielvorhersagen:", pred_test[:5])
submission = pd.DataFrame({
    "id": test["id"],
    "Tm": pred_test
})

submission.to_csv("Submissions/predictions_NN.csv", index=False)



x dtype: <dtype: 'float32'>
weights dtype: <dtype: 'float32'>
Epoch 01  Loss: 62879.6445
Epoch 02  Loss: 16652.4062
Epoch 03  Loss: 9554.9199
Epoch 04  Loss: 8207.5371
Epoch 05  Loss: 7488.8682
Epoch 06  Loss: 7045.9912
Epoch 07  Loss: 6582.2388
Epoch 08  Loss: 6197.9004
Epoch 09  Loss: 5769.0342
Epoch 10  Loss: 5336.4146
Epoch 11  Loss: 4908.9102
Epoch 12  Loss: 4550.9663
Epoch 13  Loss: 4179.8125
Epoch 14  Loss: 3874.0027
Epoch 15  Loss: 3631.7195
Epoch 16  Loss: 3419.2173
Epoch 17  Loss: 3205.6108
Epoch 18  Loss: 3088.6477
Epoch 19  Loss: 2868.0762
Epoch 20  Loss: 2751.9890

Beispielvorhersagen: [333.62216 382.12903 180.07803 192.65445 242.0797 ]


In [17]:

# ----------------------
# 2) TensorFlow Dataset
# ----------------------
batch_size = 128
ds_train = tf.data.Dataset.from_tensor_slices((X_train, y_train.astype("float32")))
ds_train = ds_train.shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# ----------------------
# 3) Besseres Modell
# ----------------------
num_features = X_train.shape[1]

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(num_features,)),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1)  # linear output
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="mse"
)

# ----------------------
# 4) Training
# ----------------------
callback = tf.keras.callbacks.EarlyStopping(
    patience=30,
    restore_best_weights=True
)

history = model.fit(
    ds_train,
    epochs=300,
    callbacks=[callback],
    verbose=1
)

# ----------------------
# 5) Predictions + CSV
# ----------------------
pred_test = model.predict(X_test).flatten()

pd.DataFrame({
    "id": test["id"],
    "Tm": pred_test
}).to_csv("Submissions/predictions_NN_improved.csv", index=False)


Epoch 1/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 79581.6016
Epoch 2/300
 8/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 76229.2686

C:\Users\hwind\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 69291.6953
Epoch 3/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 51001.1953
Epoch 4/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 29023.6270
Epoch 5/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 14675.5068
Epoch 6/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 8527.5811
Epoch 7/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 6012.7163
Epoch 8/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5241.3262
Epoch 9/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 4250.9004
Epoch 10/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 3834.4275
Epoch 11/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2855.1746
Epoch 12/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2599.3091
Epoch 13/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2167.2104
Epoch 14/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 2135.0894
Epoch 15/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 2225.0964
Epoch 16/300
21/21 ━━━

In [ ]:
# usage of optuna
#is worse than the second try
from sklearn.model_selection import train_test_split

# ----------------------------------------------------------
# Daten vorbereiten
# ----------------------------------------------------------
X_train_np = X_train.astype("float32")
y_train_np = y_train.astype("float32")

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_np, y_train_np, test_size=0.15, random_state=42
)

# TF Datasets Funktion
def make_dataset(X, y, batch_size):
    return (
        tf.data.Dataset.from_tensor_slices((X, y))
        .shuffle(len(X))
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

# ----------------------------------------------------------
# Optuna Objective
# ----------------------------------------------------------
def objective(trial):

    # Hyperparameter suchen
    n1 = trial.suggest_int("n_hidden_1", 128, 1024)
    n2 = trial.suggest_int("n_hidden_2", 64, 512)
    n3 = trial.suggest_int("n_hidden_3", 0, 256)   # 0 = kein Layer 3

    lr = trial.suggest_float("learning_rate", 1e-4, 5e-3, log=True)
    dropout = trial.suggest_float("dropout", 0.0, 0.3)
    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256, 512])
    activation = trial.suggest_categorical("activation", ["relu", "selu", "gelu"])

    # Dataset
    ds_train = make_dataset(X_tr, y_tr, batch_size)
    ds_val = make_dataset(X_val, y_val, batch_size)

    # --------------------------
    # Modell bauen
    # --------------------------
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(X_tr.shape[1],)))
    model.add(tf.keras.layers.Dense(n1, activation=activation))
    model.add(tf.keras.layers.Dropout(dropout))
    model.add(tf.keras.layers.Dense(n2, activation=activation))

    if n3 > 0:
        model.add(tf.keras.layers.Dense(n3, activation=activation))

    model.add(tf.keras.layers.Dense(1))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="mse"
    )

    # --------------------------
    # Training (kurz für Optuna)
    # --------------------------
    history = model.fit(
        ds_train,
        validation_data=ds_val,
        epochs=40,        # kurz halten, wird oft ausgeführt
        verbose=0
    )

    # Ziel: Minimale Validierungs-MSE
    val_loss = min(history.history["val_loss"])
    return val_loss

# ----------------------------------------------------------
# Optuna ausführen
# ----------------------------------------------------------
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=40)   # 40–80 empfohlen

print("\nBest Trial:")
print(study.best_trial.params)

# ----------------------------------------------------------
# FINAL MODEL MIT BESTEN PARAMETERN
# ----------------------------------------------------------
best_params = study.best_trial.params

n1 = best_params["n_hidden_1"]
n2 = best_params["n_hidden_2"]
n3 = best_params["n_hidden_3"]
lr = best_params["learning_rate"]
dropout = best_params["dropout"]
batch_size = best_params["batch_size"]
activation = best_params["activation"]

model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(X_train_np.shape[1],)))
model.add(tf.keras.layers.Dense(n1, activation=activation))
model.add(tf.keras.layers.Dropout(dropout))
model.add(tf.keras.layers.Dense(n2, activation=activation))
if n3 > 0:
    model.add(tf.keras.layers.Dense(n3, activation=activation))
model.add(tf.keras.layers.Dense(1))

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr),
    loss="mse"
)

ds_full = make_dataset(X_train_np, y_train_np, batch_size)

# ------------- FULL TRAINING ---------------
history = model.fit(
    ds_full,
    epochs=200,
    verbose=1
)

# ----------------------------------------------------------
# Vorhersagen für Testdaten
# ----------------------------------------------------------
pred_test = model.predict(X_test.astype("float32")).flatten()

pd.DataFrame({
    "id": test["id"],
    "Tm": pred_test
}).to_csv("Submissions/predictions_NN_optuna.csv", index=False)

print("Fertig! predictions_NN_optuna.csv gespeichert.")



Best Trial:
{'n_hidden_1': 890, 'n_hidden_2': 382, 'n_hidden_3': 29, 'learning_rate': 0.0006289872944239899, 'dropout': 0.09346698381557328, 'batch_size': 128, 'activation': 'gelu'}
Epoch 1/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 42332.8750
Epoch 2/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 11948.4014
Epoch 3/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 7789.4575
Epoch 4/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 6436.6538
Epoch 5/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 5085.0171
Epoch 6/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 4344.1338
Epoch 7/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 3646.9148
Epoch 8/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 3171.9226
Epoch 9/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 2863.3147
Epoch 10/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 2595.1758
Epoch 11/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 2461.6191
Epoch 12/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 